In [2]:
import requests
import fake_useragent
from typing import List
import pandas as pd
import  numpy as np

## Загрузка данных

In [3]:
def get_all_regions() -> List[str]:
    user_agent = fake_useragent.UserAgent()
    regions = requests.get(
        url="https://api.hh.ru/areas",
        headers={"user_agent": user_agent.random}
    ).json()
    return [str(region["id"]) for region in regions[0]["areas"]]

In [4]:
def get_count_vacancies(job_title: str, regions: List[str]) -> dict[str, dict[str, int]]:
    user_agent = fake_useragent.UserAgent()
    mean_salary_by_region = []
    result_row = {}
    for region in regions:
        url = f"https://api.hh.ru/vacancies?clusters=true&only_with_salary=true&enable_snippets=true&st=searchVacancy' \
            '&text={job_title}&search_field=name&area={region}"
        vacancies = requests.get(
            url=url,
            headers={"user_agent": user_agent.random}                    
        ).json()
        if "items" in vacancies.keys():
            for vacancy in vacancies["items"]:
                if vacancy["salary"]["from"] is None:
                    continue
                mean_salary_by_region.append(vacancy["salary"]["from"])
        else:
            continue
    result_row[job_title] = {
        "Средняя зарплата.": int(sum(mean_salary_by_region) / len(mean_salary_by_region)),
        "Количество вакансий в России.": len(mean_salary_by_region) * 43
    }
    return result_row

In [5]:
profs = pd.read_csv("../repositories/proffesions.csv")